# Tesseract OCR Pipeline with Layout Analysis

This notebook creates a complete OCR pipeline using **pytesseract** that:
- ✅ Reads layout_data.json for proper reading order
- ✅ Processes cropped sections based on spatial layout
- ✅ Maintains left-to-right, top-to-bottom reading order
- ✅ Generates clean markdown files
- ✅ Handles multi-page documents
- ✅ Preserves document structure (headers, tables, text)

**Reading Order Strategy:**
- Primary: Uses reading_order from layout analysis
- Secondary: Sorts by top-to-bottom (y-coordinate)
- Tertiary: Sorts by left-to-right (x-coordinate)

**Prerequisites:**
- Tesseract OCR installed on system
- Cropped sections from layout detection
- layout_data.json with bounding boxes and reading order

## 1. Import Required Libraries

In [1]:
import json
import logging
from pathlib import Path
from typing import Dict, Any, List, Tuple, Optional
import re
import warnings
warnings.filterwarnings('ignore')

# OCR and Image processing
try:
    import pytesseract
    from PIL import Image
    import numpy as np
    print("✓ pytesseract and PIL imported successfully")
except ImportError as e:
    print(f"❌ Error: {e}")
    print("Please install: pip install pytesseract pillow")
    print("Also install Tesseract: sudo apt-get install tesseract-ocr (Linux)")
    raise

# Data processing
import pandas as pd
from datetime import datetime
from tqdm.auto import tqdm

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✓ All libraries imported")

✓ pytesseract and PIL imported successfully
✓ All libraries imported


## 2. Configuration

In [2]:
# Configuration
CONFIG = {
    'input_dir': 'output_results',
    'output_dir': 'tesseract_markdown_output',
    
    # Tesseract OCR settings
    'tesseract_lang': 'eng',  # Language: eng, fra, deu, spa, etc.
    'tesseract_config': '--psm 6',  # Page Segmentation Mode: 6 = uniform block of text
    'confidence_threshold': 0.0,  # Minimum confidence (0-100), 0 = no filtering
    
    # Reading order strategy
    'use_layout_reading_order': True,  # Use reading_order from layout_data.json
    'fallback_to_spatial': True,  # If no reading_order, sort by position
    'sort_by_position': 'top_left',  # 'top_left', 'left_top', or 'reading_order'
    
    # Output settings
    'include_element_type_headers': True,  # Add type info like "## Section Header"
    'add_spacing_between_elements': True,  # Add blank lines between elements
    'format_tables': True,  # Try to format table content
    'include_confidence_comments': False,  # Add OCR confidence as HTML comments
    
    # Document structure
    'separate_pages': True,  # Add page separators in markdown
    'page_separator': '\n---\n\n',  # Separator between pages
}

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

Configuration:
  input_dir: output_results
  output_dir: tesseract_markdown_output
  tesseract_lang: eng
  tesseract_config: --psm 6
  confidence_threshold: 0.0
  use_layout_reading_order: True
  fallback_to_spatial: True
  sort_by_position: top_left
  include_element_type_headers: True
  add_spacing_between_elements: True
  format_tables: True
  include_confidence_comments: False
  separate_pages: True
  page_separator: 
---




## 3. Helper Functions - File Parsing

In [3]:
def parse_crop_filename(filename: str) -> Dict[str, Any]:
    """
    Parse crop filename to extract metadata.
    Format: page_001_order_001_type_id_27.png
    """
    parts = filename.replace('.png', '').split('_')
    
    metadata = {
        'filename': filename,
        'page': None,
        'order': None,
        'element_type': None,
        'element_id': None
    }
    
    try:
        # Extract page number
        if 'page' in parts:
            page_idx = parts.index('page')
            if page_idx + 1 < len(parts):
                metadata['page'] = int(parts[page_idx + 1])
        
        # Extract reading order
        if 'order' in parts:
            order_idx = parts.index('order')
            if order_idx + 1 < len(parts):
                metadata['order'] = int(parts[order_idx + 1])
        
        # Extract element ID
        if 'id' in parts:
            id_idx = parts.index('id')
            if id_idx + 1 < len(parts):
                metadata['element_id'] = int(parts[id_idx + 1])
        
        # Extract element type (between order and id)
        if 'order' in parts and 'id' in parts:
            order_idx = parts.index('order')
            id_idx = parts.index('id')
            if order_idx + 2 < id_idx:
                metadata['element_type'] = '_'.join(parts[order_idx + 2:id_idx])
    
    except (ValueError, IndexError) as e:
        logger.warning(f"Error parsing filename {filename}: {e}")
    
    return metadata


def load_layout_data(document_dir: Path) -> Optional[Dict]:
    """Load layout_data.json for a document."""
    layout_file = document_dir / "layout_data.json"
    
    if not layout_file.exists():
        logger.warning(f"No layout_data.json found in {document_dir}")
        return None
    
    try:
        with open(layout_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        logger.error(f"Error loading layout data: {e}")
        return None


print("✓ File parsing functions defined")

✓ File parsing functions defined


## 4. OCR Functions

In [4]:
def perform_tesseract_ocr(image_path: Path, config: Dict) -> Dict[str, Any]:
    """
    Perform OCR using Tesseract on a single image.
    Returns text and confidence score.
    """
    try:
        # Load image
        image = Image.open(image_path)
        
        # Perform OCR with detailed data
        ocr_data = pytesseract.image_to_data(
            image,
            lang=config['tesseract_lang'],
            config=config['tesseract_config'],
            output_type=pytesseract.Output.DICT
        )
        
        # Extract text and confidence
        texts = []
        confidences = []
        
        for i, text in enumerate(ocr_data['text']):
            if text.strip():  # Only non-empty text
                conf = float(ocr_data['conf'][i])
                if conf >= config['confidence_threshold']:
                    texts.append(text)
                    confidences.append(conf)
        
        # Combine text
        full_text = ' '.join(texts)
        avg_confidence = sum(confidences) / len(confidences) if confidences else 0.0
        
        return {
            'success': True,
            'text': full_text,
            'confidence': avg_confidence,
            'word_count': len(texts)
        }
    
    except Exception as e:
        logger.error(f"OCR error for {image_path.name}: {e}")
        return {
            'success': False,
            'text': '',
            'confidence': 0.0,
            'error': str(e)
        }


print("✓ OCR functions defined")

✓ OCR functions defined


## 5. Reading Order & Sorting Functions

In [5]:
def get_element_layout_info(element_id: int, layout_data: Dict, page_num: int) -> Optional[Dict]:
    """
    Get layout information for a specific element from layout_data.json.
    Returns bounding box and reading order.
    """
    if not layout_data or 'pages' not in layout_data:
        return None
    
    # Find the page
    page_idx = page_num - 1
    if page_idx >= len(layout_data['pages']):
        return None
    
    page = layout_data['pages'][page_idx]
    
    # Find the element
    for elem in page.get('elements', []):
        if elem.get('id') == element_id:
            bbox = elem.get('bounding_box', {})
            return {
                'reading_order': elem.get('reading_order'),
                'bbox': bbox,
                'left': bbox.get('left', 0),
                'top': bbox.get('top', 0),
                'right': bbox.get('right', 0),
                'bottom': bbox.get('bottom', 0),
                'type': elem.get('type', 'unknown')
            }
    
    return None


def sort_elements_by_reading_order(elements: List[Dict], config: Dict) -> List[Dict]:
    """
    Sort elements by reading order.
    Priority:
    1. Page number (ascending)
    2. Reading order from layout (if available)
    3. Top coordinate (top to bottom)
    4. Left coordinate (left to right)
    """
    def sort_key(elem: Dict) -> Tuple:
        page = elem.get('page', 0)
        
        if config['use_layout_reading_order'] and elem.get('layout_info'):
            layout = elem['layout_info']
            reading_order = layout.get('reading_order', 999)
            return (page, reading_order, layout.get('top', 0), layout.get('left', 0))
        else:
            # Fallback to spatial sorting
            order = elem.get('order', 999)
            return (page, order, 0, 0)
    
    return sorted(elements, key=sort_key)


print("✓ Reading order functions defined")

✓ Reading order functions defined


## 6. Markdown Generation Functions

## 6. Advanced Column Detection

This section implements intelligent column detection that can identify 1, 2, 3, or more columns automatically.

In [6]:
def detect_document_columns(elements: List[Dict], page_num: int) -> Dict[str, Any]:
    """
    Automatically detect the number of columns in a document (1, 2, 3, or more).
    Uses clustering on horizontal positions to identify columns.
    """
    page_elements = [e for e in elements if e.get('page') == page_num]
    
    if not page_elements:
        return {
            'num_columns': 0,
            'columns': [],
            'top_elements': [],
            'bottom_elements': [],
            'column_boundaries': [],
            'layout_type': 'empty'
        }
    
    # Get page width
    page_width = 1654.0
    elements_with_layout = [e for e in page_elements if e.get('layout_info')]
    
    if elements_with_layout:
        max_right = max([e['layout_info'].get('right', 0) for e in elements_with_layout])
        if max_right > 0:
            page_width = max_right
    
    # Collect center positions of all elements
    element_centers = []
    for elem in elements_with_layout:
        layout = elem['layout_info']
        left = layout.get('left', 0)
        right = layout.get('right', 0)
        width = right - left
        center = left + (width / 2)
        element_centers.append({
            'element': elem,
            'center': center,
            'left': left,
            'right': right,
            'width': width,
            'top': layout.get('top', 0),
            'bottom': layout.get('bottom', 0)
        })
    
    if not element_centers:
        return {
            'num_columns': 1,
            'columns': [page_elements],
            'top_elements': [],
            'bottom_elements': [],
            'column_boundaries': [0, page_width],
            'layout_type': 'single_column'
        }
    
    # Sort by center position
    element_centers.sort(key=lambda x: x['center'])
    
    # Identify full-width elements (headers, footers, tables)
    full_width_threshold = page_width * 0.7
    full_width_elements = [ec for ec in element_centers if ec['width'] > full_width_threshold]
    narrow_elements = [ec for ec in element_centers if ec['width'] <= full_width_threshold]
    
    if len(narrow_elements) < 2:
        # Only full-width elements or very few elements
        return {
            'num_columns': 1,
            'columns': [page_elements],
            'top_elements': [],
            'bottom_elements': [],
            'column_boundaries': [0, page_width],
            'layout_type': 'single_column'
        }
    
    # Cluster elements by their horizontal centers using simple gap-based clustering
    # Sort narrow elements by center position
    sorted_centers = sorted([ec['center'] for ec in narrow_elements])
    
    # Find gaps in the horizontal distribution
    gaps = []
    for i in range(len(sorted_centers) - 1):
        gap = sorted_centers[i + 1] - sorted_centers[i]
        if gap > page_width * 0.15:  # Significant gap (15% of page width)
            gaps.append({
                'position': (sorted_centers[i] + sorted_centers[i + 1]) / 2,
                'size': gap
            })
    
    # Determine number of columns based on gaps
    num_columns = len(gaps) + 1
    
    # Calculate column boundaries
    if num_columns == 1:
        column_boundaries = [0, page_width]
    else:
        column_boundaries = [0] + [gap['position'] for gap in gaps] + [page_width]
    
    # Assign elements to columns
    columns = [[] for _ in range(num_columns)]
    
    for ec in narrow_elements:
        center = ec['center']
        # Find which column this element belongs to
        for i in range(num_columns):
            left_boundary = column_boundaries[i]
            right_boundary = column_boundaries[i + 1]
            
            if left_boundary <= center < right_boundary:
                columns[i].append(ec['element'])
                break
    
    # Sort elements within each column by vertical position (top to bottom)
    for i in range(num_columns):
        columns[i] = sorted(columns[i], key=lambda x: (
            x.get('layout_info', {}).get('top', 0),
            x.get('order', 0)
        ))
    
    # Detect vertical range of multi-column section
    column_start = None
    column_end = None
    
    if num_columns > 1:
        # Find elements that appear in multiple columns at similar heights
        all_column_elements = [ec for ec in narrow_elements]
        if all_column_elements:
            column_start = min([ec['top'] for ec in all_column_elements])
            column_end = max([ec['bottom'] for ec in all_column_elements])
    
    # Categorize full-width elements as top or bottom
    top_elements = []
    bottom_elements = []
    
    for ec in full_width_elements:
        if column_start and ec['top'] < column_start - 50:
            top_elements.append(ec['element'])
        elif column_end and ec['top'] > column_end + 50:
            bottom_elements.append(ec['element'])
        else:
            # Full-width element in the middle of columns - add to bottom
            bottom_elements.append(ec['element'])
    
    # Sort top and bottom elements
    top_elements.sort(key=lambda x: (
        x.get('layout_info', {}).get('top', 0),
        x.get('order', 0)
    ))
    bottom_elements.sort(key=lambda x: (
        x.get('layout_info', {}).get('top', 0),
        x.get('order', 0)
    ))
    
    # Determine layout type
    if num_columns == 1:
        layout_type = 'single_column'
    elif num_columns == 2:
        layout_type = 'two_column'
    elif num_columns == 3:
        layout_type = 'three_column'
    else:
        layout_type = f'{num_columns}_column'
    
    return {
        'num_columns': num_columns,
        'columns': columns,
        'top_elements': top_elements,
        'bottom_elements': bottom_elements,
        'column_boundaries': column_boundaries,
        'layout_type': layout_type,
        'page_width': page_width
    }


def generate_markdown_with_columns(elements: List[Dict], config: Dict, doc_name: str) -> str:
    """
    Generate markdown document with automatic column detection and formatting.
    Supports 1, 2, 3, or more columns.
    """
    markdown_parts = []
    
    
    
    # Group elements by page
    pages = {}
    for elem in elements:
        page_num = elem.get('page', 1)
        if page_num not in pages:
            pages[page_num] = []
        pages[page_num].append(elem)
    
    # Process each page
    for page_num in sorted(pages.keys()):
        if config['separate_pages']:
            markdown_parts.append(f"## Page {page_num}\n")
        
        # Detect columns
        column_info = detect_document_columns(elements, page_num)
        
        # Add layout info comment
        markdown_parts.append(f"<!-- Layout: {column_info['layout_type']} ({column_info['num_columns']} column(s)) -->")
        markdown_parts.append('')
        
        # Process top elements (full-width headers, titles)
        for elem in column_info['top_elements']:
            formatted = format_element_for_markdown(elem, config)
            if formatted:
                markdown_parts.append(formatted)
                if config['add_spacing_between_elements']:
                    markdown_parts.append('')
        
        # Process columns
        if column_info['num_columns'] > 1:
            # Multi-column layout
            markdown_parts.append(f'<div style="display: flex; gap: 20px;">')
            markdown_parts.append('')
            
            for col_idx, column in enumerate(column_info['columns']):
                markdown_parts.append(f'<div style="flex: 1;">  <!-- Column {col_idx + 1} -->')
                markdown_parts.append('')
                
                for elem in column:
                    formatted = format_element_for_markdown(elem, config)
                    if formatted:
                        markdown_parts.append(formatted)
                        if config['add_spacing_between_elements']:
                            markdown_parts.append('')
                
                markdown_parts.append('</div>')
                markdown_parts.append('')
            
            markdown_parts.append('</div>')
            markdown_parts.append('')
        else:
            # Single column - just output elements
            for column in column_info['columns']:
                for elem in column:
                    formatted = format_element_for_markdown(elem, config)
                    if formatted:
                        markdown_parts.append(formatted)
                        if config['add_spacing_between_elements']:
                            markdown_parts.append('')
        
        # Process bottom elements (full-width tables, footers)
        for elem in column_info['bottom_elements']:
            formatted = format_element_for_markdown(elem, config)
            if formatted:
                markdown_parts.append(formatted)
                if config['add_spacing_between_elements']:
                    markdown_parts.append('')
        
        # Add page separator
        if config['separate_pages'] and page_num < max(pages.keys()):
            markdown_parts.append(config['page_separator'])
    
    return '\n'.join(markdown_parts)


print("✓ Advanced column detection functions defined")

✓ Advanced column detection functions defined


In [ ]:
def format_element_for_markdown(element: Dict, config: Dict) -> str:
    """
    Format an element for markdown output based on its type.
    """
    text = element.get('text', '').strip()
    if not text:
        return ''
    
    element_type = element.get('element_type', 'text')
    markdown_lines = []
    
    # Add confidence comment if enabled
    if config['include_confidence_comments'] and 'confidence' in element:
        confidence = element['confidence']
        markdown_lines.append(f"<!-- OCR Confidence: {confidence:.1f}% -->")
    
    # Format based on element type
    if element_type == 'title':
        markdown_lines.append(f"# {text}")
    
    elif element_type == 'section_header':
        markdown_lines.append(f"## {text}")
    
    elif element_type == 'page_header':
        markdown_lines.append(f"*{text}*")
    
    elif element_type == 'page_footer':
        markdown_lines.append(f"*{text}*")
    
    elif element_type == 'table':
        if config['format_tables']:
            # Try to format as markdown table (basic formatting)
            markdown_lines.append("```")
            markdown_lines.append(text)
            markdown_lines.append("```")
        else:
            markdown_lines.append(text)
    
    elif element_type == 'key_value_region':
        # Format key-value pairs
        markdown_lines.append(f"**{text}**")
    
    else:  # text, paragraph, list_item, etc.
        markdown_lines.append(text)
    
    return '\n'.join(markdown_lines)


def detect_document_columns(elements: List[Dict], page_num: int) -> Dict[str, Any]:
    """
    Automatically detect the number of columns in a document (1, 2, 3, or more).
    Uses clustering on horizontal positions to identify columns.
    Returns dict with column information and grouped elements.
    """
    page_elements = [e for e in elements if e.get('page') == page_num]
    
    if not page_elements:
        return {
            'left_column': [], 
            'right_column': [], 
            'top_elements': [], 
            'bottom_elements': [], 
            'has_columns': False,
            'layout_type': 'single_column',
            'num_columns': 1
        }
    
    # Get page width to determine column boundary
    page_width = 1654.0  # Default width
    if page_elements and page_elements[0].get('layout_info'):
        # Try to estimate page width from elements
        max_right = max([e['layout_info'].get('right', 0) for e in page_elements if e.get('layout_info')])
        if max_right > 0:
            page_width = max_right
    
    mid_point = page_width / 2
    threshold = page_width * 0.1  # 10% threshold for column detection
    
    # First, categorize elements by horizontal position
    left_side = []
    right_side = []
    full_width = []
    
    for elem in page_elements:
        layout = elem.get('layout_info')
        if not layout:
            full_width.append(elem)
            continue
        
        left = layout.get('left', 0)
        right = layout.get('right', 0)
        width = right - left
        
        # Check if element spans full width
        if width > page_width * 0.7:
            full_width.append(elem)
        # Check if element is in left column
        elif right < mid_point + threshold:
            left_side.append(elem)
        # Check if element is in right column
        elif left > mid_point - threshold:
            right_side.append(elem)
        else:
            full_width.append(elem)
    
    # Detect the vertical range of the two-column section
    # Find elements that are genuinely paired (similar vertical positions on both sides)
    two_column_start = None
    two_column_end = None
    
    if left_side and right_side:
        # Find pairs of elements at similar vertical positions
        paired_tops = []
        paired_bottoms = []
        vertical_threshold = 100  # Elements within 100px vertically are considered paired
        
        for left_elem in left_side:
            left_top = left_elem.get('layout_info', {}).get('top', 999)
            left_bottom = left_elem.get('layout_info', {}).get('bottom', 0)
            
            # Check if there's a right element at a similar vertical position
            for right_elem in right_side:
                right_top = right_elem.get('layout_info', {}).get('top', 999)
                right_bottom = right_elem.get('layout_info', {}).get('bottom', 0)
                
                # If tops are close, they're likely paired
                if abs(left_top - right_top) < vertical_threshold:
                    paired_tops.append(min(left_top, right_top))
                    paired_bottoms.append(max(left_bottom, right_bottom))
                    break
        
        if paired_tops and paired_bottoms:
            two_column_start = min(paired_tops)
            two_column_end = max(paired_bottoms)
    
    # Categorize elements
    top_elements = []
    bottom_elements = []
    left_column = []
    right_column = []
    
    for elem in full_width:
        top = elem.get('layout_info', {}).get('top', 0)
        if two_column_start and top < two_column_start:
            top_elements.append(elem)
        elif two_column_end and top > two_column_end:
            bottom_elements.append(elem)
        else:
            # This full-width element is in the middle of the column section
            # Add it to bottom if it's a table or large element
            if elem.get('element_type') in ['table', 'section_header']:
                bottom_elements.append(elem)
            else:
                top_elements.append(elem)
    
    # Assign left and right column elements (only within the column range)
    for elem in left_side:
        top = elem.get('layout_info', {}).get('top', 0)
        elem_type = elem.get('element_type', '')
        
        # Section headers or elements outside column range should go to top/bottom
        if two_column_end and top > two_column_end + 20:
            bottom_elements.append(elem)
        elif two_column_start and two_column_end and two_column_start <= top <= two_column_end + 20:
            left_column.append(elem)
        elif two_column_start and top < two_column_start:
            top_elements.append(elem)
        else:
            bottom_elements.append(elem)
    
    for elem in right_side:
        top = elem.get('layout_info', {}).get('top', 0)
        elem_type = elem.get('element_type', '')
        
        # Section headers or elements outside column range should go to top/bottom
        if two_column_end and top > two_column_end + 20:
            bottom_elements.append(elem)
        elif two_column_start and two_column_end and two_column_start <= top <= two_column_end + 20:
            right_column.append(elem)
        elif two_column_start and top < two_column_start:
            top_elements.append(elem)
        else:
            bottom_elements.append(elem)
    
    # Determine layout type and number of columns
    has_columns = len(left_column) > 0 and len(right_column) > 0
    layout_type = 'two_column' if has_columns else 'single_column'
    num_columns = 2 if has_columns else 1
    
    return {
        'left_column': sorted(left_column, key=lambda x: (x.get('layout_info', {}).get('top', 0), x.get('order', 0))),
        'right_column': sorted(right_column, key=lambda x: (x.get('layout_info', {}).get('top', 0), x.get('order', 0))),
        'top_elements': sorted(top_elements, key=lambda x: (x.get('layout_info', {}).get('top', 0), x.get('order', 0))),
        'bottom_elements': sorted(bottom_elements, key=lambda x: (x.get('layout_info', {}).get('top', 0), x.get('order', 0))),
        'has_columns': has_columns,
        'layout_type': layout_type,
        'num_columns': num_columns
    }


def generate_markdown_document(elements: List[Dict], config: Dict, doc_name: str) -> str:
    """
    Generate complete markdown document from sorted elements.
    Handles two-column layouts intelligently.
    """
    markdown_parts = []
    
    # Add document title
    markdown_parts.append(f"# {doc_name}")
    markdown_parts.append(f"\n*Generated with Tesseract OCR - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n")
    
    # Group elements by page
    pages = {}
    for elem in elements:
        page_num = elem.get('page', 1)
        if page_num not in pages:
            pages[page_num] = []
        pages[page_num].append(elem)
    
    # Process each page
    for page_num in sorted(pages.keys()):
        page_elements = pages[page_num]
        
        # Add page header
        if config['separate_pages']:
            markdown_parts.append(f"\n---\n## Page {page_num}\n")
        
        # Detect columns for this page
        columns = detect_document_columns(page_elements, page_num)
        
        if columns['has_columns']:
            # Process top elements first (headers at the top)
            for elem in columns['top_elements']:
                formatted = format_element_for_markdown(elem, config)
                if formatted:
                    markdown_parts.append(formatted)
                    if config['add_spacing_between_elements']:
                        markdown_parts.append('')
            
            # Create two-column section
            markdown_parts.append('<div style="display: flex; gap: 20px;">')
            markdown_parts.append('')
            
            # Left column
            markdown_parts.append('<div style="flex: 1;">')
            markdown_parts.append('')
            for elem in columns['left_column']:
                formatted = format_element_for_markdown(elem, config)
                if formatted:
                    markdown_parts.append(formatted)
                    if config['add_spacing_between_elements']:
                        markdown_parts.append('')
            markdown_parts.append('</div>')
            markdown_parts.append('')
            
            # Right column
            markdown_parts.append('<div style="flex: 1;">')
            markdown_parts.append('')
            for elem in columns['right_column']:
                formatted = format_element_for_markdown(elem, config)
                if formatted:
                    markdown_parts.append(formatted)
                    if config['add_spacing_between_elements']:
                        markdown_parts.append('')
            markdown_parts.append('</div>')
            markdown_parts.append('')
            
            markdown_parts.append('</div>')
            markdown_parts.append('')
            
            # Process bottom elements (tables, footers, etc.)
            for elem in columns['bottom_elements']:
                formatted = format_element_for_markdown(elem, config)
                if formatted:
                    markdown_parts.append(formatted)
                    if config['add_spacing_between_elements']:
                        markdown_parts.append('')
        else:
            # Single column layout - process normally
            for elem in page_elements:
                formatted = format_element_for_markdown(elem, config)
                if formatted:
                    markdown_parts.append(formatted)
                    if config['add_spacing_between_elements']:
                        markdown_parts.append('')
        
        # Add page separator
        if config['separate_pages'] and page_num < max(pages.keys()):
            markdown_parts.append(config['page_separator'])
    
    return '\n'.join(markdown_parts)


print("✓ Markdown generation functions defined")

✓ Markdown generation functions defined


## 7. Main Processing Function

In [8]:
def process_document(document_dir: Path, config: Dict) -> Dict[str, Any]:
    """
    Process a single document:
    1. Load layout_data.json
    2. Find all cropped sections
    3. Perform OCR on each section
    4. Sort by reading order
    5. Generate markdown
    """
    doc_name = document_dir.name
    logger.info(f"Processing document: {doc_name}")
    
    # Load layout data
    layout_data = load_layout_data(document_dir)
    
    # Find cropped sections
    crops_dir = document_dir / "cropped_sections"
    if not crops_dir.exists():
        return {
            'success': False,
            'error': f"No cropped_sections directory found in {document_dir}"
        }
    
    crop_files = sorted([f for f in crops_dir.glob('*.png') if f.name.startswith('page_')])
    
    if not crop_files:
        return {
            'success': False,
            'error': f"No crop files found in {crops_dir}"
        }
    
    # Process each crop
    elements = []
    
    for crop_file in tqdm(crop_files, desc=f"OCR {doc_name}", leave=False):
        # Parse filename
        metadata = parse_crop_filename(crop_file.name)
        
        # Perform OCR
        ocr_result = perform_tesseract_ocr(crop_file, config)
        
        if not ocr_result['success'] or not ocr_result['text'].strip():
            continue
        
        # Get layout information
        layout_info = None
        if layout_data and metadata['page'] and metadata['element_id'] is not None:
            layout_info = get_element_layout_info(
                metadata['element_id'],
                layout_data,
                metadata['page']
            )
        
        # Create element record
        element = {
            'page': metadata['page'],
            'order': metadata['order'],
            'element_type': metadata['element_type'],
            'element_id': metadata['element_id'],
            'text': ocr_result['text'],
            'confidence': ocr_result['confidence'],
            'word_count': ocr_result['word_count'],
            'layout_info': layout_info
        }
        
        elements.append(element)
    
    # Sort elements by reading order
    sorted_elements = sort_elements_by_reading_order(elements, config)
    
    # Generate markdown
    markdown_content = generate_markdown_document(sorted_elements, config, doc_name)
    
    return {
        'success': True,
        'document_name': doc_name,
        'total_elements': len(sorted_elements),
        'markdown': markdown_content,
        'elements': sorted_elements
    }


print("✓ Main processing function defined")

✓ Main processing function defined


In [9]:
def process_document_with_column_detection(document_dir: Path, config: Dict) -> Dict[str, Any]:
    """
    Process a single document with automatic column detection:
    1. Load layout_data.json
    2. Find all cropped sections
    3. Perform OCR on each section
    4. Sort by reading order
    5. Detect columns automatically
    6. Generate markdown with proper column formatting
    """
    doc_name = document_dir.name
    logger.info(f"Processing document: {doc_name}")
    
    # Load layout data
    layout_data = load_layout_data(document_dir)
    
    # Find cropped sections
    crops_dir = document_dir / "cropped_sections"
    if not crops_dir.exists():
        return {
            'success': False,
            'error': f"No cropped_sections directory found in {document_dir}"
        }
    
    crop_files = sorted([f for f in crops_dir.glob('*.png') if f.name.startswith('page_')])
    
    if not crop_files:
        return {
            'success': False,
            'error': f"No crop files found in {crops_dir}"
        }
    
    # Process each crop
    elements = []
    
    for crop_file in tqdm(crop_files, desc=f"OCR {doc_name}", leave=False):
        # Parse filename
        metadata = parse_crop_filename(crop_file.name)
        
        # Perform OCR
        ocr_result = perform_tesseract_ocr(crop_file, config)
        
        if not ocr_result['success'] or not ocr_result['text'].strip():
            continue
        
        # Get layout information
        layout_info = None
        if layout_data and metadata['page'] and metadata['element_id'] is not None:
            layout_info = get_element_layout_info(
                metadata['element_id'],
                layout_data,
                metadata['page']
            )
        
        # Create element record
        element = {
            'page': metadata['page'],
            'order': metadata['order'],
            'element_type': metadata['element_type'],
            'element_id': metadata['element_id'],
            'text': ocr_result['text'],
            'confidence': ocr_result['confidence'],
            'word_count': ocr_result['word_count'],
            'layout_info': layout_info
        }
        
        elements.append(element)
    
    # Sort elements by reading order
    sorted_elements = sort_elements_by_reading_order(elements, config)
    
    # Detect columns for each page
    pages = list(set([e.get('page', 1) for e in sorted_elements]))
    column_info = {}
    for page_num in pages:
        col_info = detect_document_columns(sorted_elements, page_num)
        column_info[page_num] = col_info
        logger.info(f"  Page {page_num}: {col_info['layout_type']} ({col_info['num_columns']} columns)")
    
    # Generate markdown with column formatting
    markdown_content = generate_markdown_with_columns(sorted_elements, config, doc_name)
    
    return {
        'success': True,
        'document_name': doc_name,
        'total_elements': len(sorted_elements),
        'markdown': markdown_content,
        'elements': sorted_elements,
        'column_info': column_info
    }


print("✓ Enhanced processing function with column detection defined")

✓ Enhanced processing function with column detection defined


## 8. Batch Processing Function

In [10]:
def batch_process_documents(input_dir: Path, output_dir: Path, config: Dict) -> Dict[str, Any]:
    """
    Process all documents in the input directory.
    """
    # Create output directory
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Find all document directories
    document_dirs = [d for d in input_dir.iterdir() if d.is_dir()]
    
    if not document_dirs:
        logger.warning(f"No document directories found in {input_dir}")
        return {'success': False, 'error': 'No documents found'}
    
    logger.info(f"Found {len(document_dirs)} documents to process")
    
    results = {
        'total_documents': len(document_dirs),
        'successful': 0,
        'failed': 0,
        'details': []
    }
    
    for doc_dir in tqdm(document_dirs, desc="Processing documents"):
        try:
            # Process document
            result = process_document(doc_dir, config)
            
            if result['success']:
                # Save markdown
                output_file = output_dir / f"{result['document_name']}.md"
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(result['markdown'])
                
                results['successful'] += 1
                results['details'].append({
                    'document': result['document_name'],
                    'status': 'success',
                    'elements': result['total_elements'],
                    'output_file': str(output_file)
                })
                
                logger.info(f"✓ {result['document_name']}: {result['total_elements']} elements")
            else:
                results['failed'] += 1
                results['details'].append({
                    'document': doc_dir.name,
                    'status': 'failed',
                    'error': result.get('error', 'Unknown error')
                })
                logger.error(f"✗ {doc_dir.name}: {result.get('error')}")
        
        except Exception as e:
            results['failed'] += 1
            results['details'].append({
                'document': doc_dir.name,
                'status': 'failed',
                'error': str(e)
            })
            logger.error(f"✗ {doc_dir.name}: {e}")
    
    return results


print("✓ Batch processing function defined")

✓ Batch processing function defined


## 9. Test Single Document

## 9. Test with Automatic Column Detection

In [11]:
# Test with automatic column detection
input_dir = Path(CONFIG['input_dir'])
output_dir = Path(CONFIG['output_dir'])

# Find first document
document_dirs = [d for d in input_dir.iterdir() if d.is_dir()]
if document_dirs:
    test_doc = document_dirs[0]
    print(f"Testing with enhanced column detection: {test_doc.name}\\n")
    print("="*70)
    
    # Process single document with column detection
    result = process_document_with_column_detection(test_doc, CONFIG)
    
    if result['success']:
        print(f"\\n✅ Successfully processed: {result['document_name']}")
        print(f"   Total elements: {result['total_elements']}")
        
        # Display column information
        print(f"\\n📊 Column Detection Results:")
        for page_num, col_info in result['column_info'].items():
            print(f"\\n   Page {page_num}:")
            print(f"     - Layout Type: {col_info['layout_type']}")
            print(f"     - Number of Columns: {col_info['num_columns']}")
            print(f"     - Page Width: {col_info.get('page_width', 'N/A'):.1f}px")
            if col_info['num_columns'] > 1:
                print(f"     - Column Boundaries: {[f'{b:.1f}' for b in col_info['column_boundaries']]}")
                for i, col in enumerate(col_info['columns']):
                    print(f"     - Column {i+1}: {len(col)} elements")
            print(f"     - Top Elements: {len(col_info['top_elements'])}")
            print(f"     - Bottom Elements: {len(col_info['bottom_elements'])}")
        
        # Save to file
        output_dir.mkdir(parents=True, exist_ok=True)
        output_file = output_dir / f"{result['document_name']}.md"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(result['markdown'])
        
        print(f"\\n✓ Markdown saved to: {output_file}")
        print(f"\\n📄 Markdown preview (first 600 chars):")
        print("="*70)
        print(result['markdown'][:600])
        if len(result['markdown']) > 600:
            print("...")
    else:
        print(f"✗ Error: {result.get('error')}")
else:
    print("No documents found for testing")

2025-10-26 13:19:41,699 - INFO - Processing document: batch1-0287


Testing with enhanced column detection: batch1-0287\n


OCR batch1-0287:   0%|          | 0/13 [00:00<?, ?it/s]

KeyError: 'layout_type'

In [ ]:
# Test with a single document
input_dir = Path(CONFIG['input_dir'])
output_dir = Path(CONFIG['output_dir'])

# Find first document
document_dirs = [d for d in input_dir.iterdir() if d.is_dir()]
if document_dirs:
    test_doc = document_dirs[0]
    print(f"Testing with document: {test_doc.name}\n")
    
    # Process single document
    result = process_document(test_doc, CONFIG)
    
    if result['success']:
        print(f"✓ Successfully processed: {result['document_name']}")
        print(f"  Total elements: {result['total_elements']}")
        print(f"\nFirst 5 elements:")
        
        for i, elem in enumerate(result['elements'][:5], 1):
            print(f"\n  [{i}] Page {elem['page']}, Order {elem['order']}")
            print(f"      Type: {elem['element_type']}")
            print(f"      Text: {elem['text'][:80]}..." if len(elem['text']) > 80 else f"      Text: {elem['text']}")
            print(f"      Confidence: {elem['confidence']:.1f}%")
        
        # Save to file
        output_dir.mkdir(parents=True, exist_ok=True)
        output_file = output_dir / f"{result['document_name']}.md"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(result['markdown'])
        
        print(f"\n✓ Markdown saved to: {output_file}")
        print(f"\nMarkdown preview (first 500 chars):")
        print("="*60)
        print(result['markdown'][:500])
        print("...")
    else:
        print(f"✗ Error: {result.get('error')}")
else:
    print("No documents found for testing")

2025-10-23 01:30:46,099 - INFO - Processing document: batch1-0287


Testing with document: batch1-0287



✓ Successfully processed: batch1-0287
  Total elements: 13

First 5 elements:

  [1] Page 1, Order 1
      Type: key_value_region
      Text: Invoice no: 51335214 Date of issue: 03/27/2015
      Confidence: 95.9%

  [2] Page 1, Order 2
      Type: section_header
      Text: Seller:
      Confidence: 96.0%

  [3] Page 1, Order 3
      Type: section_header
      Text: Client:
      Confidence: 96.0%

  [4] Page 1, Order 4
      Type: text
      Text: Rivera Group 90443 lan Inlet Suite 586 Lake Abigail, WV 40743
      Confidence: 95.5%

  [5] Page 1, Order 5
      Type: text
      Text: Malone, Wilson and Carson 01909 Kyle Port South Joyce, VA 45070
      Confidence: 95.7%

✓ Markdown saved to: tesseract_markdown_output/batch1-0287.md

Markdown preview (first 500 chars):
# batch1-0287

*Generated with Tesseract OCR - 2025-10-23 01:30:48*

## Page 1

**Invoice no: 51335214 Date of issue: 03/27/2015**

<div style="display: flex; gap: 20px;">

<div style="flex: 1;">

## Seller:

Rivera Group

## 10. Batch Process All Documents

In [ ]:
# Process all documents
print("Starting batch processing...\n")

input_dir = Path(CONFIG['input_dir'])
output_dir = Path(CONFIG['output_dir'])

results = batch_process_documents(input_dir, output_dir, CONFIG)

print("\n" + "="*60)
print("BATCH PROCESSING SUMMARY")
print("="*60)
print(f"Total documents: {results['total_documents']}")
print(f"Successful: {results['successful']}")
print(f"Failed: {results['failed']}")
print(f"\nOutput directory: {output_dir}")

if results['failed'] > 0:
    print("\nFailed documents:")
    for detail in results['details']:
        if detail['status'] == 'failed':
            print(f"  - {detail['document']}: {detail['error']}")

2025-10-23 01:30:48,448 - INFO - Found 61 documents to process


Starting batch processing...



Processing documents:   0%|          | 0/61 [00:00<?, ?it/s]2025-10-23 01:30:48,452 - INFO - Processing document: batch1-0287
2025-10-23 01:30:50,805 - INFO - ✓ batch1-0287: 13 elements
Processing documents:   2%|▏         | 1/61 [00:02<02:21,  2.35s/it]2025-10-23 01:30:50,806 - INFO - Processing document: 50120516-0516
2025-10-23 01:30:52,288 - INFO - ✓ 50120516-0516: 5 elements
Processing documents:   3%|▎         | 2/61 [00:03<01:48,  1.84s/it]2025-10-23 01:30:52,289 - INFO - Processing document: invoice_Anna Gayman_42837
2025-10-23 01:30:53,673 - INFO - ✓ invoice_Anna Gayman_42837: 6 elements
Processing documents:   5%|▍         | 3/61 [00:05<01:34,  1.63s/it]2025-10-23 01:30:53,674 - INFO - Processing document: batch1-0281
2025-10-23 01:30:55,629 - INFO - ✓ batch1-0281: 11 elements
Processing documents:   7%|▋         | 4/61 [00:07<01:40,  1.76s/it]2025-10-23 01:30:55,630 - INFO - Processing document: 0001139626
2025-10-23 01:30:57,629 - INFO - ✓ 0001139626: 7 elements
Processing 


BATCH PROCESSING SUMMARY
Total documents: 61
Successful: 61
Failed: 0

Output directory: tesseract_markdown_output


## 11. View Generated Markdown

In [ ]:
# View a generated markdown file
output_dir = Path(CONFIG['output_dir'])
markdown_files = list(output_dir.glob('*.md'))

if markdown_files:
    # Display first file
    sample_file = markdown_files[0]
    print(f"Viewing: {sample_file.name}\n")
    print("="*60)
    
    with open(sample_file, 'r', encoding='utf-8') as f:
        content = f.read()
        print(content[:1500])  # First 1500 characters
        if len(content) > 1500:
            print("\n... (truncated)")
            print(f"\nTotal length: {len(content)} characters")
else:
    print("No markdown files found")

Viewing: 00921466.md

# 00921466

*Generated with Tesseract OCR - 2025-10-23 01:32:07*

## Page 1

<div style="display: flex; gap: 20px;">

<div style="flex: 1;">

BORRISTON RESEARCH LABORATORIES, INC.

August 20, 1981

LORILLARD, INC. 420 English St. Greensboro, N.C. 27420

**Attention: Dr. Harry Minnemeyer Reference: Purchase Order # 312-A BRL Ref.:  2-22+222-J Invoice No.: 5-J**

## DESCRIPTION

For submission of Final Report “Cardiovascular Testing of Compound A-11 in the Beagle Dog" at $2,700.00 per compound.

</div>

<div style="flex: 1;">

## * * ® RINVOICE® * # *

**AMOUNT. $2,700.00**

</div>

</div>

**Remittance Address: ENVIRO CONTROL INC. 11140 Rockville Pike Rockville, Md. 20852 Attn: B. Belford, Accounting**

*: :*

* % ® MINVOICE® * * *

*5050 Beech Place © Temple Hills, Maryland 20031 @ 301-899-3536,*



## Summary

This pipeline provides:

### ✅ Features Implemented:
1. **Layout-aware OCR**: Reads layout_data.json for proper structure
2. **Reading order preservation**: 
   - Primary: Uses reading_order from layout analysis
   - Secondary: Top-to-bottom sorting
   - Tertiary: Left-to-right sorting
3. **Clean markdown output**: Formatted based on element types
4. **Batch processing**: Process multiple documents at once
5. **Confidence tracking**: Monitor OCR quality
6. **Error handling**: Robust error management

### 📊 Processing Flow:
```
Input: Cropped images + layout_data.json
  ↓
Parse filenames → Extract metadata
  ↓
Load layout info → Get reading order & bounding boxes
  ↓
Perform OCR → Extract text with Tesseract
  ↓
Sort elements → By reading order (top-to-bottom, left-to-right)
  ↓
Generate markdown → Clean, structured output
  ↓
Output: Markdown files
```

### 🎯 Next Steps:
1. Run cell 9 to test single document
2. Run cell 10 to process all documents
3. Run cell 11 to view generated markdown
4. Adjust CONFIG settings as needed
5. Check output in `tesseract_markdown_output/` directory